<a href="https://colab.research.google.com/github/olesia-za/python_for_ds_tasks/blob/main/OZ_Done_10_HW4_Data_visualization_with_Plotly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнє завдання: Інтерактивні візуалізації з Plotly

## Опис завдання
У цьому домашньому завданні ви будете створювати інтерактивні візуалізації з допомогою бібліотеки Plotly. Ви дізнаєтесь різницю між Plotly Express (швидкі графіки) та Graph Objects (повний контроль), та створите інтерактивний дашборд.

**Опис колонок:**
- `datetime` - дата та час
- `season` - сезон (1=весна, 2=літо, 3=осінь, 4=зима)
- `holiday` - чи є день святковим (0=ні, 1=так)
- `workingday` - чи є день робочим (0=ні, 1=так)
- `weather` - погодні умови (1=ясно, 2=туман, 3=легкий дощ, 4=сильний дощ)
- `temp` - температура в градусах Цельсія
- `atemp` - відчувається як температура
- `humidity` - вологість (%)
- `windspeed` - швидкість вітру
- `casual` - кількість випадкових користувачів
- `registered` - кількість зареєстрованих користувачів
- `count` - загальна кількість орендованих велосипедів

## Підготовка даних


In [143]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# Завантаження даних
df = pd.read_csv('../data/yulu_rental.csv')
df['datetime'] = pd.to_datetime(df['datetime'])

# Для plotly краще не встановлювати datetime як індекс
df['month'] = df['datetime'].dt.month
df['hour'] = df['datetime'].dt.hour
df['weekday'] = df['datetime'].dt.day_name()

df['month_year'] = df['datetime'].dt.to_period('M')
df['month_year']  = df.month_year.astype(str)

# Додаємо назви сезонів
season_map = {1: 'Весна', 2: 'Літо', 3: 'Осінь', 4: 'Зима'}
df['season_name'] = df['season'].map(season_map)

## Завдання 1: Базовий інтерактивний лінійний графік (Plotly Express)

**Завдання:**
Створіть інтерактивний лінійний графік динаміки оренди за часом (рівень деталізації - як в даних) з можливістю zoom та hover.

Дайте відповіді на питання.
**Питання для інтерпретації:**
1. Яка перевага інтерактивного графіка над статичним?
2. Чому на графіку є "пробіли" - ділянки, де одна пряма лінія зʼєднує два "суцільних" блоки з даними? Як би ви це могли дослідити на статичному графіку?


In [154]:
fig  = px.line(df, x='datetime', y='count', title='Кількість орендованих велосипедів по даті')
fig.update_layout(
    hovermode='x unified',
    template='plotly_white'
)
fig.show();
# 1) є zoom та hover - можна подивитися детальніше на дані при hoover
# 2) є дні коли відсутні дані щодо оренди велосипедів, - тому дані такі і згруповані блоками, статичний поденний графік + resample також показав би таку ж картину

## Завдання 2: Scatter plot з додатковими даними (Plotly Express)

**Завдання:**
Створіть scatter plot кількості орендованих велосипедів випадковими користувачами vs кількості орендованих велосипедів зареєстрованими користувачами. Розмір точок встановіть за сумарною кількістю велосипедів, які були взяті в оренду, а колір - за сезоном. В hover_data - додайте деталі, які допоможуть вам в подальшому аналізі.

Дослідіть графік. Зверніть увагу, що ви можете вмикати і вимикати окремі сезони, якщо будете клікати на колір сезону в легенді графіку.

**Дайте відповідь на питання.**
- Як ви проінтерпретуєте роздвоєність цього графіку (дві явні лінії)? Що це означає?
- Які висновки для компанії, які дає велосипеди в оренду, ви можете зробити з цього графіку? 3 основних висновки.

In [ ]:
fig = px.scatter(df,
                x='registered',
                y='casual',
                size= 'count',
                color='season_name',
                hover_data=['workingday','month_year','weather'] )
fig.update_layout(title='Залежність між зареєстрованими та незареєстрованими користувачами',
                  height=600)
fig.show();
# 1) можливо, дві явні лінії обумовлені залежністю від робочий або ні день
# 2) висновок1 - у робочі дні значно вища частка зареєстрованих користувачів, у вихідні/святкові - частка зареєстованих та випадкових користувачів стають ближчі за значеннями
#  висновок 2 - Весною рівень оренди дуже незначний
#  висновок 3 - зареєстровані користувавчі значно частіше орендують велосипеди за будь-якої погоди, випадкові- переважно вихідні та за хорошої погоди



## Завдання 3: Порівняння Plotly Express vs Graph Objects

**Завдання:**
Створіть лінійний графік помісячної динаміки оренди велосипедів двома способами - з Plotly Express та з Graph Objects.

**Дайте відповіді на питання.**
1. Як ви розумієте основну різницю між цими двома підходами?
2. Коли краще використовувати Plotly Express?
3. Коли потрібен Graph Objects?


In [157]:
fig = px.line(
        x=df['month_year'].unique(),
        y=df.groupby('month_year')['count'].mean(),
        title='Середньомісячна динаміка оренди - PX'
        # color=df['season_name'].unique()
)
fig.show();

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=df['month_year'].unique(),
        y=df.groupby('month_year')['count'].mean(),
        mode='markers+lines',  # точки + лінії
        marker=dict(
            size=10,
            color='royalblue'
        ), # це наша hover_data
        hovertemplate='Місяць: %{x}<br>Оренда: %{y:.0f}<br>', # формат відображення
        name='Rent',
    )
)

fig.update_layout(
    title='Середньомісячна динаміка оренди - GO'
)
fig.show();
# 1) px побудувати простіше і швидше ніж з go, go має велуку кількість опцій для налаштування
# 2) px краще використовувати для швидкої побудови інтерактивних графіків
# 3) go краще використовувати для побудови інтерактивного дашборду з декількома графіками та hover, який можна наповнити потрібною додатковою інфою

## Завдання 4: Дашборд з make_subplots (Graph Objects)

**Завдання:**
Створіть дашборд з 4 різними графіками в одній фігурі:
- Bar chart - середні значення загальної кількості оренд велосипедів за сезонами
- Pie chart - відсоткове співвідношення погодних умов в даних
- Line chart - середнє значення загальної кількості оренд велосипедів за годинами протягом доби
- Scatter plot - кореляція температури vs вологість

Додайте заголовок на дашборд.

**Дайте відповідь на питання**
- На ваш погляд, яка перевага об'єднання графіків в один дашборд?

In [ ]:
fig = make_subplots(
    rows=2, cols=2,
    specs=[[ {'type':'xy'}, {'type':'domain'}],
           [ {'type':'xy'}, {'type':'xy'}]
    ],
    subplot_titles=('Середнє значення кількості ореди за сезонами', 'Відсоткове співідношення погодних умов',
                    'Середнє погодинне значення кількості оренди протягом доби', 'Температура vs Вологість')
)

# Графік 1: 
fig.add_trace(
    go.Bar(
        x=df['season_name'].unique(),
        y=df.groupby('season_name')['count'].mean(),
        text=round(df.groupby('season')['count'].mean(),0),
        name='season_name',
        hovertemplate='Number: %{y:.0f}<br>Season: %{x}<extra></extra>'
    ),
    row=1, col=1
)

# Графік 2: 
fig.add_trace(
    go.Pie(
        labels=['Ясно', 'Туман', 'Легкий дощ', 'Сильний дощ'],
        values=df['weather'].value_counts(),
        name='weather',
        # hovertemplate='Weather: %{x}<br>Percent: %{y}<extra></extra>'
    ),
    row=1, col=2
)

# Графік 3: 
fig.add_trace(
    go.Scatter(
        x=df['hour'].unique(),
        y=df.groupby('hour')['count'].mean(),
        mode='lines',
        name='hour',
        hovertemplate='Hour: %{x}<br>Number: %{y:.0f}<extra></extra>'
    ),
    row=2, col=1
)
# fig.update_xaxes(tickmode='array', tickvals=df['hour'].unique())
# Графік 4: 
fig.add_trace(
    go.Scatter(
        x=df['temp'],
        y=df['humidity'],
        mode='markers',
        # showlegend=False,
        marker=dict(
            color=df['count'],
            colorscale='viridis',
            colorbar=dict(title='Number')
        ),
        hovertemplate=(
            'Temperature: %{x}<br>'
            'Humidity: %{y}<br>'
            'Number: %{marker.color:.0f}<extra></extra>'
        )
    ),
    row=2, col=2
)

fig.update_layout(
    height=800,
    showlegend=False,
    title_text="Комплексний дашборд продажів"
)

fig.show()

#дашборд з декількома графіками легко надіслати/ надати комплексну наглядну з інтерактивними опціями інфу для допомоги прийняття рішень іншій зацікавленій особі(ам) 

## Завдання 5: 3D візуалізація

**Завдання:**
Створіть 3D scatter plot для аналізу взаємозв'язку температури, швидкості вітру та загальної кількості орендованих велосипедів. Колір встановіть за сезоном, а розмір - за загальною кількість оренд також.

Дайте відповіді на питання.
**Питання для інтерпретації:**
1. Яку додаткову інформацію, на ваш погляд, дає 3D візуалізація?
2. Чи видно кластери в 3D просторі?
3. Чи ви можете зробити висновки з цієї візуалізації, чи вам було простіше побудувати кілька 2D?



In [ ]:
fig = px.scatter_3d(df.sample(1000),
                    x='temp',
                    y='windspeed',
                    z='count',
                    color='season_name',
                    size='count',
                    title='3D scatter plot')
fig.update_layout(height=700)
fig.show()
# 1) можливість побачити одночасно залежність між 3ма змінними
# 2) так , можна помітити кластери
# 3) можливо іноді доречно додатково і 3d графік дослідити але декілька 2d графіків не менш інформативні і мені звичніші для сприйняття


## Завдання 6: Експорт та збереження інтерактивних графіків

**Завдання:**
Збережіть побудований раніше дашборд в формат HTML. Також змініть вручну щось на дашборді (зум, виділення частини графіку) і збережіть його як статичне зображення через іконку фотоапарату у формат PNG. Завантажте файли з дашбордом у HTML та PNG (або посилання на них на github) разом з посиланням на цей ноутбук при здачі ДЗ.


In [170]:
fig = make_subplots(
    rows=2, cols=2,
    specs=[[ {'type':'xy'}, {'type':'domain'}],
           [ {'type':'xy'}, {'type':'xy'}]
    ],
    subplot_titles=('Середнє значення кількості ореди за сезонами', 'Відсоткове співідношення погодних умов',
                    'Середнє погодинне значення кількості оренди протягом доби', 'Температура vs Вологість')
)

# Графік 1: 
fig.add_trace(
    go.Bar(
        x=df['season_name'].unique(),
        y=df.groupby('season_name')['count'].mean(),
        text=round(df.groupby('season')['count'].mean(),0),
        name='season_name',
        hovertemplate='Number: %{y:.0f}<br>Season: %{x}<extra></extra>'
    ),
    row=1, col=1
)

# Графік 2: 
fig.add_trace(
    go.Pie(
        labels=['Ясно', 'Туман', 'Легкий дощ', 'Сильний дощ'],
        values=df['weather'].value_counts(),
        name='weather',
        # hovertemplate='Weather: %{x}<br>Percent: %{y}<extra></extra>'
    ),
    row=1, col=2
)

# Графік 3: 
fig.add_trace(
    go.Scatter(
        x=df['hour'].unique(),
        y=df.groupby('hour')['count'].mean(),
        mode='lines',
        name='hour',
        hovertemplate='Hour: %{x}<br>Number: %{y:.0f}<extra></extra>'
    ),
    row=2, col=1
)
# fig.update_xaxes(tickmode='array', tickvals=df['hour'].unique())
# Графік 4: 
fig.add_trace(
    go.Scatter(
        x=df['temp'],
        y=df['humidity'],
        mode='markers',
        # showlegend=False,
        marker=dict(
            color=df['count'],
            colorscale='viridis',
            colorbar=dict(title='Number')
        ),
        hovertemplate=(
            'Temperature: %{x}<br>'
            'Humidity: %{y}<br>'
            'Number: %{marker.color:.0f}<extra></extra>'
        )
    ),
    row=2, col=2
)

fig.update_layout(
    height=800,
    showlegend=False,
    title_text="Комплексний дашборд продажів"
)
# Зберегти як HTML
fig.write_html("dashboard.html")
fig.show()